# Image Processing Techniques for Tactile Enhancement

Welcome to the third notebook in our Art Tactile Transform series! This notebook focuses on the image processing pipeline that transforms raw depth maps into optimized tactile representations.

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Understand each step in the image processing pipeline
- Learn how different filters affect tactile quality
- Master the configuration parameters for optimal results
- Implement custom image processing workflows
- Troubleshoot common image processing issues

## 🖼️ The Image Processing Pipeline

Our application transforms depth maps through several processing stages to create optimal tactile representations:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageDraw
from scipy.ndimage import gaussian_filter
import sys
from pathlib import Path

# Add the project to Python path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / "src"))

print("🖼️ Image Processing Pipeline Overview")
print("="*45)

pipeline_stages = [
    {
        "stage": "1. Input Validation",
        "purpose": "Ensure image is in correct format",
        "operations": ["Format check", "Size validation", "Color mode conversion"]
    },
    {
        "stage": "2. Grayscale Conversion",
        "purpose": "Convert to single-channel depth representation",
        "operations": ["RGB to grayscale", "Preserve depth information"]
    },
    {
        "stage": "3. Border Addition",
        "purpose": "Add consistent edges for 3D printing",
        "operations": ["Padding with border pixels", "Edge consistency"]
    },
    {
        "stage": "4. Gaussian Blur",
        "purpose": "Smooth surface for better tactile feel",
        "operations": ["Noise reduction", "Surface smoothing", "Feature preservation"]
    },
    {
        "stage": "5. Value Clamping",
        "purpose": "Control contrast and dynamic range",
        "operations": ["Min/max limiting", "Contrast enhancement"]
    },
    {
        "stage": "6. Normalization",
        "purpose": "Scale to 0-1 range for processing",
        "operations": ["Linear scaling", "Range mapping"]
    },
    {
        "stage": "7. Height Inversion",
        "purpose": "Optionally invert depth interpretation",
        "operations": ["Conditional inversion", "Dark=high option"]
    }
]

for stage_info in pipeline_stages:
    print(f"\n📋 {stage_info['stage']}")
    print(f"   🎯 Purpose: {stage_info['purpose']}")
    print(f"   ⚙️ Operations: {', '.join(stage_info['operations'])}")

print("\n💡 Key Insight:")
print("Each stage serves both image quality and tactile usability goals.")
print("The pipeline optimizes for touch perception, not just visual appeal.")

## 🔍 Deep Dive: Process Image Function

Let's examine our main image processing function:

In [ ]:
from art_tactile_transform.main import process_image
import inspect

print("🔍 Process Image Function Analysis")
print("="*40)

# Show function signature
signature = inspect.signature(process_image)
print(f"📋 Function Signature:")
print(f"process_image{signature}")

# Show function source
print("\n📝 Function Implementation:")
print(inspect.getsource(process_image))

print("\n🔧 Parameter Details:")
params = {
    "gaussian_blur_radius": "Smoothing filter radius (0 = disabled)",
    "clamp_min/max": "Contrast control range (0-255)",
    "border_pixels": "Edge padding size in pixels",
    "invert_heights": "Reverse depth interpretation (bool)"
}

for param, description in params.items():
    print(f"• {param:<20}: {description}")

## 🎨 Interactive Processing Demonstration

Let's create a demonstration of each processing stage:

In [ ]:
def create_demo_depth_map(size=(128, 128)):
    """Create a synthetic depth map for demonstration."""
    img = Image.new('RGB', size, 'white')
    draw = ImageDraw.Draw(img)
    
    # Create interesting depth features
    center_x, center_y = size[0] // 2, size[1] // 2
    
    # Background gradient
    for y in range(size[1]):
        intensity = int(255 * (1 - y / size[1]))
        draw.line([(0, y), (size[0], y)], fill=(intensity, intensity, intensity))
    
    # Add some geometric shapes with different depths
    draw.ellipse([20, 20, 60, 60], fill=(200, 200, 200))  # Light circle
    draw.rectangle([80, 30, 110, 70], fill=(100, 100, 100))  # Dark rectangle
    draw.ellipse([center_x-15, center_y-15, center_x+15, center_y+15], fill=(50, 50, 50))  # Dark center
    
    # Add some noise
    for _ in range(20):
        x, y = np.random.randint(0, size[0]), np.random.randint(0, size[1])
        intensity = np.random.randint(0, 255)
        draw.point((x, y), fill=(intensity, intensity, intensity))
    
    return img

def demonstrate_processing_stages(input_image):
    """Show the effect of each processing stage."""
    stages = {}
    
    # Stage 1: Original
    stages['Original'] = input_image.copy()
    
    # Stage 2: Grayscale conversion
    gray_img = input_image.convert('L')
    stages['Grayscale'] = gray_img
    
    # Stage 3: With border
    border_img = process_image(gray_img, border_pixels=5)
    stages['With Border'] = border_img
    
    # Stage 4: Gaussian blur
    blur_img = process_image(gray_img, gaussian_blur_radius=3)
    stages['Blurred'] = blur_img
    
    # Stage 5: Clamped
    clamp_img = process_image(gray_img, clamp_min=50, clamp_max=200)
    stages['Clamped'] = clamp_img
    
    # Stage 6: Inverted
    invert_img = process_image(gray_img, invert_heights=True)
    stages['Inverted'] = invert_img
    
    return stages

# Create demo image
demo_img = create_demo_depth_map()
processing_stages = demonstrate_processing_stages(demo_img)

print("🎨 Processing Stage Demonstration")
print("="*35)
print(f"✅ Created demo depth map with {len(processing_stages)} processing variations")

# Display the stages if matplotlib is available
try:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (stage_name, img) in enumerate(processing_stages.items()):
        if i < len(axes):
            # Convert PIL to numpy for display
            if img.mode == 'RGB':
                img_array = np.array(img)
            else:
                img_array = np.array(img)
                if len(img_array.shape) == 2:
                    # For grayscale images
                    pass
            
            axes[i].imshow(img_array, cmap='gray' if img.mode == 'L' else None)
            axes[i].set_title(stage_name)
            axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(processing_stages), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle("Image Processing Stages Comparison", y=1.02)
    plt.show()
    
    print("📊 Stage comparison visualized above")
except ImportError:
    print("📊 Install matplotlib to see stage visualizations")

print("\n🔍 Observe the differences:")
print("• Border adds consistent edges")
print("• Blur smooths out noise and sharp edges")
print("• Clamping adjusts contrast range")
print("• Inversion changes depth interpretation")

## 🔧 Parameter Effects Analysis

Let's explore how different parameter values affect the final result:

In [ ]:
def analyze_parameter_effects(base_image):
    """Analyze the effect of different parameter values."""
    
    print("🔧 Parameter Effects Analysis")
    print("="*35)
    
    # Convert to grayscale for processing
    if base_image.mode != 'L':
        base_image = base_image.convert('L')
    
    # Test Gaussian blur radius
    print("\n📊 Gaussian Blur Radius Effects:")
    blur_tests = [0, 1, 3, 5, 10]
    blur_results = {}
    
    for radius in blur_tests:
        result = process_image(base_image, gaussian_blur_radius=radius)
        blur_results[f"Radius {radius}"] = result
        
        # Calculate some statistics
        img_array = np.array(result)
        mean_val = np.mean(img_array)
        std_val = np.std(img_array)
        
        print(f"   Radius {radius:2d}: Mean={mean_val:6.2f}, Std={std_val:6.2f}")
    
    # Test clamping effects
    print("\n📊 Clamping Range Effects:")
    clamp_tests = [
        (0, 255),    # No clamping
        (50, 200),   # Moderate clamping
        (100, 150),  # Heavy clamping
        (0, 128),    # Lower half only
        (128, 255)   # Upper half only
    ]
    
    clamp_results = {}
    for min_val, max_val in clamp_tests:
        result = process_image(base_image, clamp_min=min_val, clamp_max=max_val)
        clamp_results[f"Clamp {min_val}-{max_val}"] = result
        
        img_array = np.array(result)
        actual_min, actual_max = np.min(img_array), np.max(img_array)
        dynamic_range = actual_max - actual_min
        
        print(f"   {min_val:3d}-{max_val:3d}: Range={actual_min:5.1f}-{actual_max:5.1f}, Dynamic={dynamic_range:5.1f}")
    
    # Test border effects
    print("\n📊 Border Size Effects:")
    border_tests = [0, 2, 5, 10, 20]
    border_results = {}
    
    for border_size in border_tests:
        result = process_image(base_image, border_pixels=border_size)
        border_results[f"Border {border_size}"] = result
        
        original_size = base_image.size
        new_size = result.size
        size_increase = (new_size[0] - original_size[0], new_size[1] - original_size[1])
        
        print(f"   Border {border_size:2d}: Size {original_size} → {new_size} (+{size_increase})")
    
    return {
        'blur': blur_results,
        'clamp': clamp_results,
        'border': border_results
    }

# Analyze parameter effects
parameter_results = analyze_parameter_effects(demo_img)

print("\n💡 Parameter Selection Guidelines:")
print("• Blur Radius: 2-5 for smooth tactile surfaces, 0 for sharp details")
print("• Clamping: Narrow range for high contrast, wide for subtle variations")
print("• Border: 5-10 pixels for stable 3D printing edges")
print("• Inversion: Use when dark areas should be raised (artistic preference)")

## 🧠 Understanding Tactile Perception

The image processing choices directly affect how the final tactile model feels:

In [ ]:
print("🧠 Tactile Perception & Image Processing")
print("="*45)

tactile_considerations = {
    "Surface Smoothness": {
        "goal": "Comfortable touch experience",
        "processing": "Gaussian blur reduces sharp edges",
        "parameter": "gaussian_blur_radius=2-5",
        "trade_off": "Smoothness vs detail preservation"
    },
    "Height Variation": {
        "goal": "Distinguishable tactile features",
        "processing": "Clamping controls contrast",
        "parameter": "clamp_min/max for dynamic range",
        "trade_off": "Contrast vs gradual transitions"
    },
    "Edge Definition": {
        "goal": "Clear tactile boundaries",
        "processing": "Border pixels provide consistent edges",
        "parameter": "border_pixels=5-10",
        "trade_off": "Edge clarity vs size increase"
    },
    "Depth Interpretation": {
        "goal": "Intuitive height mapping",
        "processing": "Inversion changes dark=high vs light=high",
        "parameter": "invert_heights boolean",
        "trade_off": "Artistic choice vs conventional mapping"
    }
}

for aspect, details in tactile_considerations.items():
    print(f"\n👆 {aspect}")
    for key, value in details.items():
        print(f"   {key.replace('_', ' ').title():<12}: {value}")

print("\n🔬 Human Touch Sensitivity:")
touch_facts = [
    "Fingertips can detect height differences as small as 10 micrometers",
    "Optimal tactile features are 0.5-3mm in height variation",
    "Sharp edges can be uncomfortable and may break during printing",
    "Gradual slopes are easier to interpret than sudden height changes",
    "Pattern regularity helps with tactile interpretation"
]

for fact in touch_facts:
    print(f"• {fact}")

print("\n🎯 Optimization Strategy:")
print("1. Start with moderate blur (radius=3) for smooth surfaces")
print("2. Use clamping to ensure adequate height variation")
print("3. Add borders for stable printing and handling")
print("4. Test both normal and inverted height interpretations")
print("5. Consider the intended use case (art, education, accessibility)")

## 🔍 Advanced Processing Techniques

Let's explore some advanced image processing concepts that could enhance our pipeline:

In [ ]:
def demonstrate_advanced_techniques(base_image):
    """Demonstrate advanced image processing techniques."""
    
    print("🔍 Advanced Processing Techniques")
    print("="*40)
    
    if base_image.mode != 'L':
        base_image = base_image.convert('L')
    
    img_array = np.array(base_image).astype(float)
    advanced_results = {}
    
    # 1. Histogram Equalization
    def histogram_equalization(img_array):
        """Apply histogram equalization for better contrast."""
        hist, bins = np.histogram(img_array.flatten(), 256, [0, 256])
        cdf = hist.cumsum()
        cdf_normalized = cdf * hist.max() / cdf.max()
        
        # Create mapping
        cdf_m = np.ma.masked_equal(cdf, 0)
        cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
        cdf = np.ma.filled(cdf_m, 0).astype('uint8')
        
        # Apply mapping
        equalized = cdf[img_array.astype('uint8')]
        return equalized
    
    hist_eq = histogram_equalization(img_array)
    advanced_results['Histogram Equalized'] = Image.fromarray(hist_eq.astype('uint8'), mode='L')
    
    # 2. Adaptive Gaussian Filter
    def adaptive_gaussian(img_array):
        """Apply variable blur based on local image properties."""
        # Calculate local variance
        from scipy.ndimage import uniform_filter
        local_mean = uniform_filter(img_array, size=5)
        local_var = uniform_filter(img_array**2, size=5) - local_mean**2
        
        # Normalize variance to 0-1
        var_norm = (local_var - local_var.min()) / (local_var.max() - local_var.min() + 1e-8)
        
        # Apply variable blur: more blur where variance is low
        result = img_array.copy()
        for sigma in [1, 2, 3]:
            blurred = gaussian_filter(img_array, sigma=sigma)
            mask = var_norm < (sigma / 3.0)
            result[mask] = blurred[mask]
        
        return result
    
    adaptive_blur = adaptive_gaussian(img_array)
    advanced_results['Adaptive Blur'] = Image.fromarray(adaptive_blur.astype('uint8'), mode='L')
    
    # 3. Edge-Preserving Smoothing (simplified bilateral filter)
    def simple_edge_preserving(img_array, sigma_spatial=2, sigma_intensity=20):
        """Simple edge-preserving smoothing."""
        result = img_array.copy()
        h, w = img_array.shape
        
        for i in range(1, h-1):
            for j in range(1, w-1):
                center_val = img_array[i, j]
                weights = []
                values = []
                
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        neighbor_val = img_array[i+di, j+dj]
                        
                        # Spatial weight (distance)
                        spatial_weight = np.exp(-(di**2 + dj**2) / (2 * sigma_spatial**2))
                        
                        # Intensity weight (similarity)
                        intensity_weight = np.exp(-((center_val - neighbor_val)**2) / (2 * sigma_intensity**2))
                        
                        total_weight = spatial_weight * intensity_weight
                        weights.append(total_weight)
                        values.append(neighbor_val)
                
                # Weighted average
                weights = np.array(weights)
                values = np.array(values)
                result[i, j] = np.sum(weights * values) / np.sum(weights)
        
        return result
    
    # Apply to a smaller region for demonstration (full image would be slow)
    small_region = img_array[20:60, 20:60]
    edge_preserving = simple_edge_preserving(small_region)
    
    # Create full-size result for display
    edge_result = img_array.copy()
    edge_result[20:60, 20:60] = edge_preserving
    advanced_results['Edge Preserving'] = Image.fromarray(edge_result.astype('uint8'), mode='L')
    
    # 4. Multi-scale Processing
    def multiscale_enhancement(img_array):
        """Combine multiple scales for enhanced detail."""
        # Create multiple scales
        scale1 = gaussian_filter(img_array, sigma=1)  # Fine details
        scale2 = gaussian_filter(img_array, sigma=3)  # Medium features
        scale3 = gaussian_filter(img_array, sigma=6)  # Large structures
        
        # Combine scales with different weights
        result = 0.5 * scale1 + 0.3 * scale2 + 0.2 * scale3
        
        # Normalize to 0-255 range
        result = (result - result.min()) / (result.max() - result.min()) * 255
        
        return result
    
    multiscale = multiscale_enhancement(img_array)
    advanced_results['Multiscale'] = Image.fromarray(multiscale.astype('uint8'), mode='L')
    
    return advanced_results

# Demonstrate advanced techniques
advanced_results = demonstrate_advanced_techniques(demo_img)

print(f"✅ Generated {len(advanced_results)} advanced processing examples")

# Display advanced results
try:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, (technique_name, img) in enumerate(advanced_results.items()):
        if i < len(axes):
            axes[i].imshow(np.array(img), cmap='gray')
            axes[i].set_title(technique_name)
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle("Advanced Image Processing Techniques", y=1.02)
    plt.show()
    
    print("📊 Advanced techniques visualized above")
except ImportError:
    print("📊 Install matplotlib to see advanced technique visualizations")

print("\n🎯 When to Use Advanced Techniques:")
print("• Histogram Equalization: Low contrast images")
print("• Adaptive Blur: Images with varying detail levels")
print("• Edge Preserving: Maintain sharp boundaries while smoothing")
print("• Multiscale: Combine fine details with large structures")

## 🛠️ Custom Processing Pipeline Builder

Let's create a flexible pipeline for custom image processing:

In [ ]:
class TactileProcessingPipeline:
    """Flexible image processing pipeline for tactile enhancement."""
    
    def __init__(self):
        self.steps = []
        self.results = {}
    
    def add_step(self, name, function, **kwargs):
        """Add a processing step to the pipeline."""
        self.steps.append({
            'name': name,
            'function': function,
            'kwargs': kwargs
        })
        return self
    
    def process(self, input_image):
        """Execute the full pipeline on an input image."""
        current_image = input_image.copy()
        self.results = {'input': current_image}
        
        print("🛠️ Executing Processing Pipeline")
        print("="*35)
        
        for i, step in enumerate(self.steps, 1):
            print(f"📋 Step {i}: {step['name']}")
            
            try:
                current_image = step['function'](current_image, **step['kwargs'])
                self.results[step['name']] = current_image
                
                # Basic statistics
                if hasattr(current_image, 'size'):
                    size = current_image.size
                    mode = current_image.mode
                    print(f"   ✅ Size: {size}, Mode: {mode}")
                
            except Exception as e:
                print(f"   ❌ Error: {e}")
                break
        
        return current_image
    
    def get_results(self):
        """Return all intermediate results."""
        return self.results
    
    def clear(self):
        """Clear the pipeline steps."""
        self.steps = []
        self.results = {}
        return self

# Define processing functions compatible with the pipeline
def convert_to_grayscale(image):
    """Convert image to grayscale."""
    return image.convert('L') if image.mode != 'L' else image

def apply_blur(image, radius=3):
    """Apply Gaussian blur."""
    return image.filter(ImageFilter.GaussianBlur(radius=radius))

def add_border(image, pixels=5, fill_value=0):
    """Add border around image."""
    width, height = image.size
    new_image = Image.new(image.mode, 
                         (width + 2 * pixels, height + 2 * pixels), 
                         fill_value)
    new_image.paste(image, (pixels, pixels))
    return new_image

def apply_contrast(image, factor=1.5):
    """Adjust image contrast."""
    from PIL import ImageEnhance
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(factor)

def resize_image(image, size=(128, 128)):
    """Resize image to target size."""
    return image.resize(size, Image.Resampling.LANCZOS)

# Example pipeline configurations
def create_smooth_pipeline():
    """Pipeline optimized for smooth tactile surfaces."""
    return (TactileProcessingPipeline()
            .add_step("Grayscale", convert_to_grayscale)
            .add_step("Heavy Blur", apply_blur, radius=5)
            .add_step("Border", add_border, pixels=8)
            .add_step("Resize", resize_image, size=(64, 64)))

def create_detailed_pipeline():
    """Pipeline optimized for detailed tactile features."""
    return (TactileProcessingPipeline()
            .add_step("Grayscale", convert_to_grayscale)
            .add_step("Contrast Boost", apply_contrast, factor=1.8)
            .add_step("Light Blur", apply_blur, radius=1)
            .add_step("Border", add_border, pixels=5)
            .add_step("Resize", resize_image, size=(128, 128)))

def create_artistic_pipeline():
    """Pipeline for artistic/creative tactile representations."""
    return (TactileProcessingPipeline()
            .add_step("Grayscale", convert_to_grayscale)
            .add_step("High Contrast", apply_contrast, factor=2.5)
            .add_step("Medium Blur", apply_blur, radius=3)
            .add_step("Large Border", add_border, pixels=12)
            .add_step("Resize", resize_image, size=(96, 96)))

# Test the pipelines
print("🛠️ Custom Processing Pipeline Framework")
print("="*45)

pipelines = {
    "Smooth": create_smooth_pipeline(),
    "Detailed": create_detailed_pipeline(),
    "Artistic": create_artistic_pipeline()
}

pipeline_results = {}
for name, pipeline in pipelines.items():
    print(f"\n🎯 Testing {name} Pipeline:")
    result = pipeline.process(demo_img)
    pipeline_results[name] = result
    print(f"   Final result: {result.size}, {result.mode}")

print("\n💡 Pipeline Usage Tips:")
print("• Smooth: For accessibility and comfort")
print("• Detailed: For educational or technical models")
print("• Artistic: For creative and expressive pieces")
print("• Custom: Build your own for specific requirements")

## 🎯 Hands-On Exercise: Parameter Optimization

Your turn to experiment with different processing parameters:

In [ ]:
print("🎯 Exercise: Parameter Optimization")
print("="*40)

exercise_challenges = [
    {
        "challenge": "Tactile Comfort Optimization",
        "goal": "Create the smoothest possible tactile surface",
        "constraints": ["Must preserve main features", "Eliminate sharp edges"],
        "suggested_params": "Try gaussian_blur_radius 3-8, moderate clamping",
        "success_criteria": "Smooth gradients, no sudden height changes"
    },
    {
        "challenge": "Maximum Detail Preservation",
        "goal": "Retain as much detail as possible while ensuring printability",
        "constraints": ["Minimal blur", "High contrast", "Sharp features"],
        "suggested_params": "Try gaussian_blur_radius 0-2, wide clamping range",
        "success_criteria": "Fine details visible, crisp boundaries"
    },
    {
        "challenge": "Accessibility Optimization",
        "goal": "Optimize for visually impaired users",
        "constraints": ["Clear tactile differences", "Intuitive height mapping"],
        "suggested_params": "Try strategic clamping, test both inversion modes",
        "success_criteria": "Distinct tactile regions, logical depth"
    },
    {
        "challenge": "3D Printing Optimization",
        "goal": "Ensure successful 3D printing",
        "constraints": ["Stable base", "No overhangs", "Reasonable size"],
        "suggested_params": "Try larger borders, moderate heights",
        "success_criteria": "Printable geometry, stable structure"
    }
]

for i, challenge in enumerate(exercise_challenges, 1):
    print(f"\n🏆 Challenge {i}: {challenge['challenge']}")
    print(f"   🎯 Goal: {challenge['goal']}")
    print(f"   ⚠️  Constraints: {', '.join(challenge['constraints'])}")
    print(f"   💡 Suggested: {challenge['suggested_params']}")
    print(f"   ✅ Success: {challenge['success_criteria']}")

print("\n📋 Exercise Template:")
template_code = '''
# Create your test image
test_img = create_demo_depth_map(size=(256, 256))

# Test different parameter combinations
from art_tactile_transform.main import process_image

# Example: Smooth surface optimization
smooth_result = process_image(
    test_img.convert('L'),
    gaussian_blur_radius=5,
    clamp_min=50,
    clamp_max=200,
    border_pixels=10,
    invert_heights=False
)

# Analyze your results
img_array = np.array(smooth_result)
print(f"Height range: {img_array.min():.1f} - {img_array.max():.1f}")
print(f"Standard deviation: {img_array.std():.1f}")
'''
print(template_code)

print("\n📊 Evaluation Metrics:")
metrics = [
    "Visual inspection of processed image",
    "Height range and distribution analysis",
    "Edge sharpness measurement",
    "Surface smoothness assessment",
    "Final STL file quality check"
]

for metric in metrics:
    print(f"• {metric}")

print("\n🎲 Bonus Challenge:")
print("Create a custom processing function that automatically")
print("optimizes parameters based on image content analysis!")

## 🔧 Troubleshooting Common Issues

Let's address common image processing problems and their solutions:

In [ ]:
print("🔧 Troubleshooting Image Processing Issues")
print("="*45)

common_issues = {
    "Low Contrast Output": {
        "symptoms": ["Flat-looking result", "Poor tactile variation", "Minimal height differences"],
        "causes": ["Narrow clamping range", "Over-blurring", "Poor source depth map"],
        "solutions": ["Widen clamp_min/max range", "Reduce blur radius", "Try different AI model"]
    },
    "Too Noisy/Rough": {
        "symptoms": ["Sharp jagged edges", "Uncomfortable tactile feel", "Printing artifacts"],
        "causes": ["No blur applied", "High-frequency noise in depth map", "Inappropriate clamping"],
        "solutions": ["Increase blur radius", "Apply histogram equalization", "Use edge-preserving filter"]
    },
    "Lost Detail": {
        "symptoms": ["Important features disappeared", "Over-smoothed result", "Blob-like appearance"],
        "causes": ["Excessive blur", "Over-aggressive clamping", "Too low resolution"],
        "solutions": ["Reduce blur radius", "Adjust clamp range", "Increase target resolution"]
    },
    "Inverted Depth": {
        "symptoms": ["Foreground appears sunken", "Background raised", "Counterintuitive tactile feel"],
        "causes": ["Wrong invert_heights setting", "Depth model interpretation", "Lighting confusion"],
        "solutions": ["Toggle invert_heights", "Try different AI model", "Manually invert image"]
    },
    "Size Issues": {
        "symptoms": ["Model too large/small", "Printing time excessive", "Detail loss on resize"],
        "causes": ["Border pixels too large", "Inappropriate resolution", "Poor resize algorithm"],
        "solutions": ["Adjust border_pixels", "Change target resolution", "Use better resampling"]
    }
}

for issue, details in common_issues.items():
    print(f"\n❌ {issue}")
    print(f"   🔍 Symptoms: {', '.join(details['symptoms'])}")
    print(f"   🧠 Causes: {', '.join(details['causes'])}")
    print(f"   🔧 Solutions: {', '.join(details['solutions'])}")

print("\n🎯 Debugging Workflow:")
debug_steps = [
    "1. Check original depth map quality",
    "2. Test with minimal processing (no blur, no clamping)",
    "3. Add processing steps one by one",
    "4. Compare with known good results",
    "5. Visualize intermediate steps",
    "6. Test with different source images"
]

for step in debug_steps:
    print(step)

print("\n🛡️ Prevention Tips:")
prevention_tips = [
    "Start with conservative parameters",
    "Always visualize intermediate results",
    "Keep notes on parameter combinations",
    "Test with diverse image types",
    "Validate on actual 3D prints when possible"
]

for tip in prevention_tips:
    print(f"• {tip}")

## 📚 Key Concepts Summary

After completing this notebook, you should understand:

### ✅ Image Processing Pipeline
- **Processing Stages**: Grayscale conversion, blurring, clamping, normalization
- **Parameter Effects**: How each setting affects the final tactile result
- **Trade-offs**: Balancing smoothness, detail, and printability
- **Tactile Considerations**: Optimizing for human touch perception

### ✅ Advanced Techniques
- **Histogram Equalization**: Improving contrast distribution
- **Adaptive Processing**: Variable processing based on image content
- **Edge Preservation**: Maintaining boundaries while smoothing
- **Multi-scale Processing**: Combining different detail levels

### ✅ Practical Skills
- **Parameter Tuning**: Systematic optimization approaches
- **Custom Pipelines**: Building flexible processing workflows
- **Troubleshooting**: Identifying and solving common issues
- **Quality Assessment**: Evaluating processing results

## 🎯 Next Steps

Continue your learning journey with:

- **04_3d_modeling_stl_generation.ipynb** - Learn how processed images become 3D models
- **05_hands_on_exercises.ipynb** - Practice with comprehensive coding challenges
- **06_advanced_challenges.ipynb** - Tackle complex enhancement projects

## 🔬 Challenge Questions

Test your understanding:

1. **How would you optimize processing for a photograph vs. a line drawing?**
2. **What processing approach would work best for creating braille-like tactile patterns?**
3. **How might you automatically detect optimal blur radius for a given image?**
4. **What considerations are important when processing images for children vs. adults?**
5. **How would you modify the pipeline for different 3D printer capabilities?**

---
*Ready to explore 3D modeling and STL generation? Let's dive deeper! 🚀*